#Krzysztof Nalepa
##Sprawozdanie z ćwiczenia 3
##Podstawowe metody interpolacji

### Zadanie 1
Stablicuj następujące funkcje: sqrt(x), sin(x), x^3+2x w czterech punktach należących do przedziału 0 do 10.

In [84]:
from tabulate import tabulate
import math as m
def function_to_table(function, function_string):
  domain = [0.0, 2.5, 5.0, 7.5, 10.0]
  values = []
  for number in domain:
    values.append(function(number))
  
  print(tabulate({"x": domain, function_string: values}, headers="keys", floatfmt=".10f"))
  return domain, values

sqrt_domain, sqrt_values = function_to_table(m.sqrt, "sqrt(x)")
print("\n")
sin_domain, sin_values = function_to_table(m.sin, "sin(x)")
print("\n")
lambda_domain, lambda_values = function_to_table(lambda x: x**3 + 2 * x, "x^3+2x")

            x       sqrt(x)
-------------  ------------
 0.0000000000  0.0000000000
 2.5000000000  1.5811388301
 5.0000000000  2.2360679775
 7.5000000000  2.7386127875
10.0000000000  3.1622776602


            x         sin(x)
-------------  -------------
 0.0000000000   0.0000000000
 2.5000000000   0.5984721441
 5.0000000000  -0.9589242747
 7.5000000000   0.9379999768
10.0000000000  -0.5440211109


            x           x^3+2x
-------------  ---------------
 0.0000000000     0.0000000000
 2.5000000000    20.6250000000
 5.0000000000   135.0000000000
 7.5000000000   436.8750000000
10.0000000000  1020.0000000000


### Zadanie 2 
Napisz algorytm znajdujący wielomian interpolujący Lagrange dla powyższych stablicowanych funkcji.

In [85]:
from sympy import symbols, simplify
def lagrange_polynominal_eq(domain: table, values: table):
  if len(domain) != len(values):
    return
  
  x = symbols('x')
  y = 0
  for k in range (len(domain)):
    i = 1
    for j in range(len(domain)):
      if k != j:
        i = i * ((x - domain[j]) / (domain[k] - domain[j]))
    y += i * values[k]
  return simplify(y)

print(f"sqrt: {lagrange_polynominal_eq(sqrt_domain, sqrt_values)}\n")
print(f"sin: {lagrange_polynominal_eq(sin_domain, sin_values)}\n")
print(f"lambda: {lagrange_polynominal_eq(lambda_domain, lambda_values)}\n")


  

sqrt: x*(-0.000747009008291158*x**3 + 0.0194592721406771*x**2 - 0.187359671555827*x + 0.990906275798561)

sin: x*(-0.0132730189252085*x**3 + 0.258937302364272*x**2 - 1.53380467478385*x + 2.66293332553089)

lambda: x*(-2.22044604925031e-16*x**3 + 1.0*x**2 + 2.0)



### Zadanie 3
Porównaj wartość dokładną z wynikiem interpolacji dla punktów znajdujących się pomiędzy węzłami wielomianu (w połowie odległości) interpolującego. Oszacuj dokładność interpolacji. 

In [86]:
import matplotlib.pyplot as plt
import numpy as np

def lagrange_polynominal(domain, values, x):
  if len(domain) != len(values):
    return

  y = 0
  for k in range (len(domain)):
    i = 1
    for j in range(len(domain)):
      if k != j:
        i = i * ((x - domain[j]) / (domain[k] - domain[j]))
    y += i * values[k]
  return simplify(y)


def compare(function_name, function_original, function_interpolation, domain):
  
  values_original = []
  values_interpoloation = []
  relative_error = []
  absolute_error = []
  for number in domain:
    values_original.append(function_original(number))
    values_interpoloation.append(function_interpolation(number))
    relative_error.append(
        abs(function_original(number) - function_interpolation(number)) / function_original(number))
    absolute_error.append(abs(function_original(number) - function_interpolation(number)))
  
  print(tabulate({"x": domain, function_name: values_original, function_name + " interpolation": values_interpoloation, 
                  "realtive error": relative_error, "absolute error": absolute_error}, headers="keys", floatfmt=".10f"))

domain = [1.25, 3.75, 6.25, 8.75]
sqrt_approx = lambda x: lagrange_polynominal(sqrt_domain, sqrt_values, x)
compare("sqrt", m.sqrt, sqrt_approx, domain)

print("\n")
sin_approx = lambda x: lagrange_polynominal(sin_domain, sin_values, x)
compare("sin", m.sin, sin_approx, domain)

print("\n")
lambda_approx = lambda x: lagrange_polynominal(lambda_domain, lambda_values, x)
compare("lambda", lambda x: x**3 + 2 * x, lambda_approx, domain)


           x          sqrt    sqrt interpolation    realtive error    absolute error
------------  ------------  --------------------  ----------------  ----------------
1.2500000000  1.1180339887          0.9820659964      0.1216134695      0.1359679924
3.7500000000  1.9364916731          1.9596017579      0.0119339965      0.0231100848
6.2500000000  2.5000000000          2.4853806275      0.0058477490      0.0146193725
8.7500000000  2.9580398915          2.9830674779      0.0084608685      0.0250275864


           x            sin    sin interpolation    realtive error    absolute error
------------  -------------  -------------------  ----------------  ----------------
1.2500000000   0.9489846194         1.4054289399      0.4809817896      0.4564443205
3.7500000000  -0.5715613187        -0.5530228042     -0.0324348655      0.0185385145
6.2500000000  -0.0331792165        -0.3068165894     -8.2472523859      0.2736373728
8.7500000000   0.6247239538         1.5325091963      1.4530981

### Zadanie 4
Powtórz powyższe kroki dla 3, 5 i 8 węzłów interpolacji - podsumuj badania. 

In [87]:
def function_to_table(function, function_string, domain):
  values = []
  for number in domain:
    values.append(function(number))
  
  print(tabulate({"x": domain, function_string: values}, headers="keys", floatfmt=".10f"))
  return domain, values

domain = [0, 3.3333333333, 6.6666666666, 10]
points = [1.6666666666, 5, 8.3333333333]
sqrt_domain, sqrt_values = function_to_table(m.sqrt, "sqrt(x)", domain)
print("\n")

sqrt_approx = lambda x: lagrange_polynominal(sqrt_domain, sqrt_values, x)
compare("sqrt", m.sqrt, sqrt_approx, points)

print("\n")

lagrange_polynominal_eq(domain, sqrt_values)

            x       sqrt(x)
-------------  ------------
 0.0000000000  0.0000000000
 3.3333333333  1.8257418583
 6.6666666666  2.5819888975
10.0000000000  3.1622776602


           x          sqrt    sqrt interpolation    realtive error    absolute error
------------  ------------  --------------------  ----------------  ----------------
1.6666666666  1.2909944487          1.1024038155      0.1460816764      0.1885906332
5.0000000000  2.2360679775          2.2817061964      0.0204100320      0.0456382189
8.3333333333  2.8867513459          2.8382820294      0.0167902638      0.0484693165




x*(0.00402091444268843*x**2 - 0.0883364112925316*x + 0.79750043467331)

In [88]:
domain = [0, 3.3333333333, 6.6666666666, 10]
points = [1.6666666666, 5, 8.3333333333]
sin_domain, sin_values = function_to_table(m.sin, "sin(x)", domain)
print("\n")

sin_approx = lambda x: lagrange_polynominal(sin_domain, sin_values, x)
compare("sin", m.sin, sin_approx, points)

print("\n")

lagrange_polynominal_eq(domain, sin_values)


            x         sin(x)
-------------  -------------
 0.0000000000   0.0000000000
 3.3333333333  -0.1905679628
 6.6666666666   0.3741512305
10.0000000000  -0.5440211109


           x            sin    sin interpolation    realtive error    absolute error
------------  -------------  -------------------  ----------------  ----------------
1.6666666666   0.9954079578        -0.3295810441      1.3311014761      1.3249890019
5.0000000000  -0.9589242747         0.1372669075     -1.1431467647      1.0961911822
8.3333333333   0.8872941081         0.2403126698      0.7291623289      0.6469814383




x*(-0.0100718041092337*x**2 + 0.134705963120781*x - 0.394281331373377)

In [90]:
domain = [0, 3.3333333333, 6.6666666666, 10]
points = [1.6666666666, 5, 8.3333333333]
lambda_domain, lambda_values = function_to_table(lambda x: x**3 + 2 * x, "lambda(x)", domain)
print("\n")

lambda_approx = lambda x: lagrange_polynominal(lambda_domain, lambda_values, x)
compare("lambda", lambda x: x**3 + 2 * x, lambda_approx, points)

print("\n")

lagrange_polynominal_eq(domain, lambda_values)

            x        lambda(x)
-------------  ---------------
 0.0000000000     0.0000000000
 3.3333333333    43.7037037025
 6.6666666666   309.6296296206
10.0000000000  1020.0000000000


           x          lambda    lambda interpolation    realtive error    absolute error
------------  --------------  ----------------------  ----------------  ----------------
1.6666666666    7.9629629623            7.9629629623      0.0000000000      0.0000000000
5.0000000000  135.0000000000          135.0000000000      0.0000000000      0.0000000000
8.3333333333  595.3703703634          595.3703703634      0.0000000000      0.0000000000




x*(1.0*x**2 - 7.105427357601e-15*x + 2.00000000000003)

In [91]:
domain = [0, 2, 4, 6, 8, 10]
points = [1, 3, 5, 7, 9]
sqrt_domain, sqrt_values = function_to_table(m.sqrt, "sqrt(x)", domain)
print("\n")

sqrt_approx = lambda x: lagrange_polynominal(sqrt_domain, sqrt_values, x)
compare("sqrt", m.sqrt, sqrt_approx, points)

print("\n")

lagrange_polynominal_eq(domain, sqrt_values)

  x       sqrt(x)
---  ------------
  0  0.0000000000
  2  1.4142135624
  4  2.0000000000
  6  2.4494897428
  8  2.8284271247
 10  3.1622776602


  x          sqrt    sqrt interpolation    realtive error    absolute error
---  ------------  --------------------  ----------------  ----------------
  1  1.0000000000          0.8944129014      0.1055870986      0.1055870986
  3  1.7320508076          1.7458526511      0.0079684981      0.0138018436
  5  2.2360679775          2.2298604579      0.0027760872      0.0062075196
  7  2.6457513111          2.6521102940      0.0024034696      0.0063589830
  9  3.0000000000          2.9849436742      0.0050187753      0.0150563258




x*(0.000152632103160074*x**4 - 0.00468385585455278*x**3 + 0.0553624438949276*x**2 - 0.322895942414569*x + 1.16647762362184)

In [92]:
domain = [0, 2, 4, 6, 8, 10]
points = [1, 3, 5, 7, 9]
sin_domain, sin_values = function_to_table(m.sin, "sin(x)", domain)
print("\n")

sin_approx = lambda x: lagrange_polynominal(sin_domain, sin_values, x)
compare("sin", m.sin, sin_approx, points)

print("\n")

lagrange_polynominal_eq(domain, sin_values)

  x         sin(x)
---  -------------
  0   0.0000000000
  2   0.9092974268
  4  -0.7568024953
  6  -0.2794154982
  8   0.9893582466
 10  -0.5440211109


  x            sin    sin interpolation    realtive error    absolute error
---  -------------  -------------------  ----------------  ----------------
  1   0.8414709848         1.4133655930      0.6796367534      0.5718946082
  3   0.1411200081        -0.0839243079      1.5947016941      0.2250443159
  5  -0.9589242747        -0.7989498211     -0.1668269933      0.1599744536
  7   0.6569865987         0.4729880066      0.2800644526      0.1839985921
  9   0.4121184852         0.7803760692      0.8935721088      0.3682575839




x*(0.000997277281565665*x**4 - 0.0357554009431133*x**3 + 0.427647172081667*x**2 - 2.00632974849062*x + 3.02680629310726)

In [93]:
domain = [0, 2, 4, 6, 8, 10]
points = [1, 3, 5, 7, 9]
lambda_domain, lambda_values = function_to_table(lambda x: x**3 + 2 * x, "lambda(x)", domain)
print("\n")

lambda_approx = lambda x: lagrange_polynominal(lambda_domain, lambda_values, x)
compare("lambda", lambda x: x**3 + 2 * x, lambda_approx, points)

print("\n")

lagrange_polynominal_eq(domain, lambda_values)

  x    lambda(x)
---  -----------
  0            0
  2           12
  4           72
  6          228
  8          528
 10         1020


  x    lambda    lambda interpolation    realtive error    absolute error
---  --------  ----------------------  ----------------  ----------------
  1         3            3.0000000000      0.0000000000      0.0000000000
  3        33           33.0000000000      0.0000000000      0.0000000000
  5       135          135.0000000000      0.0000000000      0.0000000000
  7       357          357.0000000000      0.0000000000      0.0000000000
  9       747          747.0000000000      0.0000000000      0.0000000000




x*(x**2 + 2)

In [94]:
domain = [0, 1.25, 2.5, 3.75, 5, 6.25, 7.5, 8.75, 10]
points = [0.625, 1.875, 3.125, 4.375, 5.625, 6.875, 8.125, 9.375]
sqrt_domain, sqrt_values = function_to_table(m.sqrt, "sqrt(x)", domain)
print("\n")

sqrt_approx = lambda x: lagrange_polynominal(sqrt_domain, sqrt_values, x)
compare("sqrt", m.sqrt, sqrt_approx, points)

print("\n")

lagrange_polynominal_eq(domain, sqrt_values)

            x       sqrt(x)
-------------  ------------
 0.0000000000  0.0000000000
 1.2500000000  1.1180339887
 2.5000000000  1.5811388301
 3.7500000000  1.9364916731
 5.0000000000  2.2360679775
 6.2500000000  2.5000000000
 7.5000000000  2.7386127875
 8.7500000000  2.9580398915
10.0000000000  3.1622776602


           x          sqrt    sqrt interpolation    realtive error    absolute error
------------  ------------  --------------------  ----------------  ----------------
0.6250000000  0.7905694150          0.7283871006      0.0786550975      0.0621823144
1.8750000000  1.3693063938          1.3740887282      0.0034925232      0.0047823344
3.1250000000  1.7677669530          1.7666175872      0.0006501795      0.0011493658
4.3750000000  2.0916500663          2.0921829621      0.0002547729      0.0005328958
5.6250000000  2.3717082451          2.3712888349      0.0001768389      0.0004194102
6.8750000000  2.6220221204          2.6225656382      0.0002072896      0.0005435178
8.12500000

x*(-1.71451895288814e-6*x**7 + 7.76884119407205e-5*x**6 - 0.00148740448987239*x**5 + 0.0156758749680836*x**4 - 0.099322564388661*x**3 + 0.388694674726764*x**2 - 0.941629646351892*x + 1.6240980400927)

In [95]:
domain = [0, 1.25, 2.5, 3.75, 5, 6.25, 7.5, 8.75, 10]
points = [0.625, 1.875, 3.125, 4.375, 5.625, 6.875, 8.125, 9.375]
sin_domain, sin_values = function_to_table(m.sin, "sin(x)", domain)
print("\n")

sin_approx = lambda x: lagrange_polynominal(sin_domain, sin_values, x)
compare("sin", m.sin, sin_approx, points)

print("\n")

lagrange_polynominal_eq(domain, sin_values)

            x         sin(x)
-------------  -------------
 0.0000000000   0.0000000000
 1.2500000000   0.9489846194
 2.5000000000   0.5984721441
 3.7500000000  -0.5715613187
 5.0000000000  -0.9589242747
 6.2500000000  -0.0331792165
 7.5000000000   0.9379999768
 8.7500000000   0.6247239538
10.0000000000  -0.5440211109


           x            sin    sin interpolation    realtive error    absolute error
------------  -------------  -------------------  ----------------  ----------------
0.6250000000   0.5850972729         0.5931719178      0.0138005170      0.0080746449
1.8750000000   0.9540857816         0.9536543206      0.0004522246      0.0004314610
3.1250000000   0.0165918922         0.0162424864      0.0210588271      0.0003494058
4.3750000000  -0.9436221923        -0.9430559505     -0.0006000726      0.0005662419
5.6250000000  -0.6116822500        -0.6125820554     -0.0014710340      0.0008998054
6.8750000000   0.5578680081         0.5597496306      0.0033728812      0.0018816225

x*(-1.40296953938155e-5*x**7 + 0.000532426522627708*x**6 - 0.00761076285712041*x**5 + 0.0490143389632716*x**4 - 0.126259133200228*x**3 + 0.0612070660353259*x**2 - 0.219931176375359*x + 1.08666363912973)

In [96]:
domain = [0, 1.25, 2.5, 3.75, 5, 6.25, 7.5, 8.75, 10]
points = [0.625, 1.875, 3.125, 4.375, 5.625, 6.875, 8.125, 9.375]
lambda_domain, lambda_values = function_to_table(lambda x: x**3 + 2 * x, "lambda(x)", domain)
print("\n")

lambda_approx = lambda x: lagrange_polynominal(lambda_domain, lambda_values, x)
compare("lambda", lambda x: x**3 + 2 * x, lambda_approx, points)

print("\n")

lagrange_polynominal_eq(domain, lambda_values)

            x        lambda(x)
-------------  ---------------
 0.0000000000     0.0000000000
 1.2500000000     4.4531250000
 2.5000000000    20.6250000000
 3.7500000000    60.2343750000
 5.0000000000   135.0000000000
 6.2500000000   256.6406250000
 7.5000000000   436.8750000000
 8.7500000000   687.4218750000
10.0000000000  1020.0000000000


           x          lambda    lambda interpolation    realtive error    absolute error
------------  --------------  ----------------------  ----------------  ----------------
0.6250000000    1.4941406250            1.4941406250      0.0000000000      0.0000000000
1.8750000000   10.3417968750           10.3417968750      0.0000000000      0.0000000000
3.1250000000   36.7675781250           36.7675781250      0.0000000000      0.0000000000
4.3750000000   92.4902343750           92.4902343750      0.0000000000      0.0000000000
5.6250000000  189.2285156250          189.2285156250      0.0000000000      0.0000000000
6.8750000000  338.7011718750      

x*(2.77555756156289e-18*x**7 + 1.77635683940025e-16*x**6 - 7.1054273576009e-15*x**5 + 6.82121026329696e-14*x**4 - 1.81898940354586e-13*x**3 + 0.999999999999272*x**2 - 1.81898940354586e-13*x + 1.99999999999964)

Im więcej węzłów interpolacji tym dokładniej wielomian odzwierciedla funkcje. W przypadku funkcji x^3 + 2x błąd jest tak niewielki że można go pominąć. Wynika to z tego że jest to wielomian przez co oszacowanie go przez wielomian Lagrange'a jest bardzo dokładne